In [4]:
import utils
from dotenv import load_dotenv, find_dotenv
import os 

# the function for getting the openai api key 
def get_openai_api_key():
    _ = load_dotenv(find_dotenv())
    
    return os.getenv("OPENAI_API_KEY")

In [5]:
find_dotenv()

'c:\\Users\\enesm\\OneDrive\\Masaüstü\\VSCODE\\BORUSAN\\.env'

In [6]:
load_dotenv(find_dotenv())
os.getenv("OPENAI_API_KEY")

'usk-NwaUeTFbzbnpdSmPbMTWT3BlbkFJzCHpZxM9Z9xqLD0dY5bO'

In [7]:
get_openai_api_key()

'usk-NwaUeTFbzbnpdSmPbMTWT3BlbkFJzCHpZxM9Z9xqLD0dY5bO'

#### For reading the input file 

In [8]:
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=["Main_Data.pdf"]
).load_data()

#### Concetanating every page together into a one large text

In [9]:
from llama_index import Document
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2) # the large language model that we want to use
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5" #The embedding we wanna use is from Beijing Academy of Artificial Intelligence
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context) # Indexing the vectors into the vector store

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=10) #how many chunks the engine will give to model as the context

# Testing the model 

In [14]:
response = query_engine.query("Can you tell me about the performance of the 2023 BMW 320i?")
print(str(response))

The performance of the 2023 BMW 320i is exhilarating. It is equipped with a two-liter four-cylinder turbo petrol engine that generates 135 kilowatts of power and 300 newton meters of torque. This engine delivers impressive acceleration, allowing the car to go from 0 to 100 kilometers per hour in just 7.4 seconds. The 320i also offers excellent fuel efficiency, with a combined cycle rating of 6.5 liters per 100 kilometers. Its power is seamlessly transmitted to the rear wheels through an 8-speed automatic transmission.


In [ ]:
response = query_engine.query("What are the new features in the Mini Countryman 2021 compared to its previous year's model?")
print(str(response))

In [ ]:
response = query_engine.query("What are some unique features of Jaguar cars?")
print(str(response))

In [ ]:
response = query_engine.query("Can you explain the infotainment system in the Jaguar F-Pace 2022?")
print(str(response))

In [ ]:
response = query_engine.query("What maintenance tips do you have for a BMW 320i owner?")
print(str(response))

In [ ]:
# We asked this to show that our model is not hallucinating and can say "i don't know"
response = query_engine.query("What are the common reliability issues reported for the 2023 BMW 320i?") 
print(str(response))

In [ ]:
# This is an important question because we are really want to give insight to our firms about their cars potential improvements
response = query_engine.query("Are there any common complaints about the interior space or comfort level in the Mini Countryman 2021?") 
print(str(response))

In [ ]:
# This is one is also an important question. It shows that we can give suggestions to the users about the car and their potential topics that the customers may not be very satisfied if they don't know that specific information
response = query_engine.query("What performance limitations have been noted in the reviews for the 2022 BMW 320i?")
print(str(response))

In [ ]:
response = query_engine.query("Have there been any significant recalls or manufacturing issues with the Jaguar F-Pace 2022?")
print(str(response))

In [ ]:
response = query_engine.query("How environmentally friendly is the Mini Countryman 2021, especially in terms of emissions and fuel consumption?")
print(str(response))

# Evaluation setup using TruLens

#### Reading the pre-defined evaluation questions

In [16]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

"Can you tell me about the performance of the 2023 BMW 320i?"
"What are the new features in the Mini Countryman 2021 compared to its previous year's model?"
"What are some unique features of Jaguar cars?"
"Can you explain the infotainment system in the Jaguar F-Pace 2022?"
"What maintenance tips do you have for a BMW 320i owner?"
"Are there any common complaints about the interior space or comfort level in the Mini Countryman 2021?
"What performance limitations have been noted in the reviews for the 2022 BMW 320i?"
"How environmentally friendly is the Mini Countryman 2021, especially in terms of emissions and fuel consumption?"


In [17]:
from trulens_eval import Tru
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [18]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [ ]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)
        print(response)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
# launches on http://localhost:8501/
tru.run_dashboard()

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()